In [ ]:
%matplotlib inline

# system
import os
import importlib
import datetime

# scipy
import numpy as np # type: ignore
from scipy import signal # type: ignore

# matplotlib
import matplotlib.pyplot as plt # type: ignore

# grandlib
import grand.dataio.root_trees as rt # type: ignore

#import tools
import database.tools as tools

plt.style.use('tableau-colorblind10')
plt.style.use('/pbs/home/p/pcorrea/tools/matplotlib_style_sans-serif.txt')

savefig  = True
plot_dir = '/pbs/home/p/pcorrea/grand/nutrig/plots'

colors = ['b','m','r']

In [ ]:
# root_file = '/sps/grand/data/gp13/GrandRoot/2024/02/GP13_20240208_142053_RUN125_MD_RAW_10s_ChanXYZ_20dB_12DUs_test_103_dat.root'
#root_file  = '/sps/grand/data/gp13/GrandRoot/2024/03/GP13_20240319_161913_RUN125_MD_RAW_10s_ChanXYZ_20dB_6DUs_test_067.root'
root_file = '/sps/grand/data/gp13/GrandRoot/2024/03/GP13_20240314_041623_RUN125_MD_RAW_10s_ChanXYZ_20dB_11DUs_test_362_dat.root'

df   = rt.DataFile(root_file)
tadc = df.tadc

n_traces = tadc.get_number_of_entries()
traces   = np.zeros( (n_traces,3,1024) )

for i in range(n_traces):
    tadc.get_entry(i)
    traces[i] = np.array(tadc.trace_ch[0][1:])

In [ ]:
window = signal.windows.general_gaussian(traces.shape[-1],10,traces.shape[-1]/2.3)
plt.plot(window)
plt.xlabel('ADC samples of 2 ns')
plt.ylabel('Windowing [a.u.]')
plt.show()

In [ ]:
fft  = np.fft.rfft(traces)
freq = np.fft.rfftfreq(1024,d=1/500)

In [ ]:
importlib.reload(tools)

#traces_filtered = tools.filter_traces(traces,freq_highpass=43,freqs_notch=[118.9,132.8,137.8],bw_notch=[3,2.5,1.]) #freqs_notch=[119,133,138] bw_notch=[2.5,2.5,1.]
traces_filtered = tools.filter_traces(traces)
fft_filtered    = np.fft.rfft(traces_filtered)

In [ ]:
fft_mean          = np.mean(np.abs(fft),axis=0)
fft_mean_filtered = np.mean(np.abs(fft_filtered),axis=0)

for ch in range(fft_mean.shape[0])[:]:
    plt.plot(freq,fft_mean[ch],color=colors[ch],alpha=.3)
    plt.plot(freq,fft_mean_filtered[ch],label=f'Channel {ch}',color=colors[ch])

plt.yscale('log')
plt.xlabel('Frequency [MHz]')
plt.ylabel('Mean FFT [a.u.]')
plt.legend()
plt.title(os.path.basename(root_file),fontsize=14)
#plt.xlim([110,120])
plt.show()

In [ ]:
rms = np.sqrt( np.mean( traces**2,axis=2 ) )
bins = np.arange(0,100)

for ch in range(rms.shape[1]):
    median = np.median(rms[:,ch])
    plt.hist(rms[:,ch],bins=bins,alpha=.5,label=f'Channel {ch} median = {np.round(median,decimals=2)}',color=colors[ch])

plt.yscale('log')
plt.xlabel('RMS [ADC counts]')
plt.ylabel('Counts')
plt.title(os.path.basename(root_file),fontsize=14)
plt.legend()
plt.show()

In [ ]:
rms_filtered = np.sqrt( np.mean( traces_filtered**2,axis=2 ) )
bins         = np.arange(0,100)

for ch in range(rms_filtered.shape[1]):
    median = np.median(rms_filtered[:,ch])
    print(np.mean(rms_filtered[:,ch]),np.std(rms_filtered[:,ch]))
    plt.hist(rms_filtered[:,ch],bins=bins,alpha=.5,label=f'Channel {ch} median = {np.round(median,decimals=2)}',color=colors[ch])

plt.yscale('log')
plt.xlabel('RMS [ADC counts]')
plt.ylabel('Counts')
plt.title(os.path.basename(root_file),fontsize=14)
plt.legend()
plt.show()

In [ ]:
idx = 400#399

tr      = traces[idx]
tr_filt = traces_filtered[idx]


fig, ax = plt.subplots(3,1,sharex=True)

ax[0].plot(tr[0],color='b',label='X')
ax[1].plot(tr[1],color='m',label='Y')
ax[2].plot(tr[2],color='r',label='Z')

ax[2].set_xlabel('ADC samples')
ax[1].set_ylabel('ADC counts')

ax[0].legend(frameon=True,framealpha=.8)
ax[1].legend(frameon=True,framealpha=.8)
ax[2].legend(frameon=True,framealpha=.8)

plt.suptitle('Original trace',va='top')

plt.show()


window = signal.windows.general_gaussian(traces.shape[-1],10,traces.shape[-1]/2.3)
fig, ax = plt.subplots(3,1,sharex=True)

ax[0].plot(tr[0]*window,color='b',label='X')
ax[1].plot(tr[1]*window,color='m',label='Y')
ax[2].plot(tr[2]*window,color='r',label='Z')

ax[2].set_xlabel('ADC samples')
ax[1].set_ylabel('ADC counts')

ax[0].legend(frameon=True,framealpha=.8)
ax[1].legend(frameon=True,framealpha=.8)
ax[2].legend(frameon=True,framealpha=.8)

plt.suptitle('Windowed trace',va='top')

plt.show()


fig, ax = plt.subplots(3,1,sharex=True)

ax[0].plot(tr_filt[0],color='b',label='X')
ax[1].plot(tr_filt[1],color='m',label='Y')
ax[2].plot(tr_filt[2],color='r',label='Z')

ax[2].set_xlabel('ADC samples')
ax[1].set_ylabel('ADC counts')

ax[0].legend(frameon=True,framealpha=.8)
ax[1].legend(frameon=True,framealpha=.8)
ax[2].legend(frameon=True,framealpha=.8)

plt.suptitle('Filtered trace',va='top')

plt.show()

In [ ]:
fig, ax = plt.subplots()

fft_idx          = np.abs(fft[idx])
fft_filtered_idx = np.abs(fft_filtered[idx])

ax.plot(freq,fft_filtered_idx[0],color='b',label='X',alpha=.5)
# ax.plot(freq,fft_filtered_idx[1],color='m',label='Y',alpha=.5)
# ax.plot(freq,fft_filtered_idx[2],color='r',label='Z',alpha=.5)

ax.plot(freq,fft_idx[0],color='b',label='X',alpha=.2)
# ax.plot(freq,fft_idx[1],color='m',label='Y',alpha=.2)
# ax.plot(freq,fft_idx[2],color='r',label='Z',alpha=.2)

ax.set_yscale('log')

ax.set_xlabel('Frequency [MHz]')
ax.set_ylabel('FFT [a.u.]')

print(freq[np.argmax(fft_idx[0])])

plt.show()

## Test threshold trigger

In [ ]:
importlib.reload(tools)

thresh1 = 35
thresh2 = 25 # 60% of th1

for i, trace in enumerate(traces_filtered[:300]):
    # thresh1 = 3*rms_filtered[i,0]
    # thresh2 = 2*rms_filtered[i,0]
    print(i)
    print(tools.thresh_trigger(trace,threshold1=thresh1,threshold2=thresh2,samples_from_trace_edge=0))

In [ ]:
idx     = 205
#tr_filt = traces_filtered[idx]
tr_filt = np.abs( traces_filtered[idx] )

fig, ax = plt.subplots(3,1,sharex=True)

ax[0].plot(tr_filt[0],color='b',label='X')
ax[1].plot(tr_filt[1],color='m',label='Y')
ax[2].plot(tr_filt[2],color='r',label='Z')

ax[0].axhline(thresh1,color='k',alpha=.7)
ax[0].axhline(thresh2,color='k',alpha=.4)
ax[1].axhline(thresh1,color='k',alpha=.7)
ax[1].axhline(thresh2,color='k',alpha=.4)

# ax[0].axvline(452,color='g',ls=':',alpha=1)
# ax[0].axvline(452+25,color='orange',ls=':',alpha=1)

ax[2].set_xlabel('ADC samples')
ax[1].set_ylabel('ADC counts')

ax[0].legend(frameon=True,framealpha=.8)
ax[1].legend(frameon=True,framealpha=.8)
ax[2].legend(frameon=True,framealpha=.8)

#plt.xlim([350,500])

plt.show()

In [ ]:
f = np.load('/sps/grand/pcorrea/nutrig/datasets/sig/rfv2_thresh_30/sig_traces_with_noise_zhaires_proton_rfv2_thresh_30_files_8000_8500_seed_21.npz')

traces_sig = f['traces']
snr_sig    = f['snr']

In [ ]:
snr_sig[:,1].max()
print(snr_sig[4413])

In [ ]:
importlib.reload(tools)

thresh1 = 35+1
thresh2 = 25+1 # 60% of th1

for i,trace in enumerate(traces_sig[0:1000]):
    if snr_sig[i,0]>=4 and snr_sig[i,0]<4.2:
        print(i,snr_sig[i])
        print(tools.thresh_trigger(trace,thresh1=thresh1,thresh2=thresh2))

In [ ]:
idx    = 147
tr_sig = traces_sig[idx]
tr_sig = np.abs( traces_sig[idx] )

fig, ax = plt.subplots(3,1,sharex=True)

ax[0].plot(tr_sig[0],color='b',label='X')
ax[1].plot(tr_sig[1],color='m',label='Y')
ax[2].plot(tr_sig[2],color='r',label='Z')

ax[0].axhline(thresh1,color='k',alpha=.7)
ax[0].axhline(thresh2,color='k',alpha=.4)
ax[1].axhline(thresh1,color='k',alpha=.7)
ax[1].axhline(thresh2,color='k',alpha=.4)

# ax[0].axvline(469,color='g',ls=':',alpha=1)
# ax[0].axvline(469+25,color='orange',ls=':',alpha=1)

ax[2].set_xlabel('ADC samples')
ax[1].set_ylabel('ADC counts')

ax[0].legend(frameon=True,framealpha=.8)
ax[1].legend(frameon=True,framealpha=.8)
ax[2].legend(frameon=True,framealpha=.8)

#plt.xlim([600,700])

plt.show()

### Estimate threshold for a given trigger rate

In [ ]:
trace_length = 1024*2 # [ns]
trigger_rate = 1e3 # [Hz]

data_taking_fraction_MD = trace_length / 1e10 # for 10s = 10^10 ns forced-trigger MD data

trigger_rate_MD = trigger_rate * data_taking_fraction_MD

trigger_rate_MD

In [ ]:
#root_file = '/sps/grand/data/gp13/GrandRoot/2024/02/GP13_20240208_142053_RUN125_MD_RAW_10s_ChanXYZ_20dB_12DUs_test_103_dat.root'
#root_file  = '/sps/grand/data/gp13/GrandRoot/2024/03/GP13_20240319_161913_RUN125_MD_RAW_10s_ChanXYZ_20dB_6DUs_test_067.root'
root_file = '/sps/grand/data/gp13/GrandRoot/2024/03/GP13_20240314_041623_RUN125_MD_RAW_10s_ChanXYZ_20dB_11DUs_test_362_dat.root'

df   = rt.DataFile(root_file)
tadc = df.tadc

n_traces = tadc.get_number_of_entries()
traces   = np.zeros( (n_traces,3,1024) )
du_ids   = np.zeros( (n_traces) )

for i in range(n_traces):
    tadc.get_entry(i)

    traces[i] = np.array(tadc.trace_ch[0][1:])
    du_ids[i] = tadc.du_id[0]

In [ ]:
tadc.get_entry(0)
start_time = datetime.datetime.fromtimestamp(tadc.gps_time[0],tz=datetime.timezone(datetime.timedelta(hours=8)))
tadc.get_entry(tadc.get_number_of_entries()-1)
end_time   = datetime.datetime.fromtimestamp(tadc.gps_time[0],tz=datetime.timezone(datetime.timedelta(hours=8)))

end_time-start_time

In [ ]:
fft  = np.fft.rfft(traces)
freq = np.fft.rfftfreq(traces.shape[-1],d=1/500)
rms  = np.sqrt( np.mean(traces**2,axis=2) )

psd_norm  = 1. / traces.shape[-1]**2 / np.diff(freq)[0] # [MHz^-1]

In [ ]:
traces_filtered = tools.filter_traces(traces)#tools.filter_traces(traces,freq_highpass=43,freqs_notch=[118.9,132.8,137.8],bw_notch=[3,2.5,1.]) #freqs_notch=[119,133,138] bw_notch=[2.5,2.5,1.]
fft_filtered    = np.fft.rfft(traces_filtered)
rms_filtered    = np.sqrt( np.mean(traces_filtered**2,axis=2) )

colors   = ['b','m','r']
channels = ['X','Y','Z']

In [ ]:
mask_dict = {du : np.zeros(du_ids.shape,dtype=bool) for du in tadc.get_list_of_dus()}

for i, du in enumerate(du_ids):
    mask_dict[du][i] = True

In [ ]:
for du in mask_dict.keys():
    mean_fft_du          = np.mean( np.abs( fft[ mask_dict[du] ]**2 ),axis=0 )
    mean_fft_du_filtered = np.mean( np.abs( fft_filtered[ mask_dict[du] ]**2 ),axis=0 )

    for ch in range(mean_fft_du.shape[0]):
        plt.plot(freq,psd_norm*mean_fft_du[ch]*1e-12,alpha=.2,label=f'{channels[ch]} raw',color=colors[ch])
        
    for ch in range(mean_fft_du.shape[0]):
        plt.plot(freq,psd_norm*mean_fft_du_filtered[ch]*1e-12,alpha=.5,label=f'{channels[ch]} filtered',color=colors[ch])

    plt.axvspan(0,50,color='grey',alpha=.3)

    plt.yscale('log')
    plt.ylim(1e-16,1e-8)
    
    plt.xlabel('Frequency [MHz]')
    plt.ylabel(r'Mean PSD [$\mathrm{V^2~ MHz^{-1}}$]')
    # title = r'\texttt{' + os.path.basename(root_file) + r'}' + f': DU {du}'
    # plt.title(title,fontsize=13)

    plt.text(151,4e-16,'GRAND preliminary', color='crimson')

    title = f'DU {du} spectrum on {start_time.date()} between {start_time.time()}--{end_time.time()} UTC+8'
    plt.title(title,fontsize=20)

    plt.legend(frameon=True,framealpha=.8,fontsize=22)

    if savefig:
        plot_name = f'psd_du_{du}_{start_time.date()}_{start_time.time()}' #+ param_str
        
        plt.savefig( os.path.join(plot_dir,plot_name+'.png') )
        plt.savefig( os.path.join(plot_dir,plot_name+'.pdf') )

    plt.show()

In [ ]:
bins = np.arange(0,100)

for du in mask_dict.keys():
    rms_du = rms[ mask_dict[du] ]

    for ch in range(rms_du.shape[1]):
        median = np.median(rms_du[:,ch])
        plt.hist(rms_du[:,ch],bins=bins,label=f'{channels[ch]} raw: median = {np.round(median,decimals=2)}',color=colors[ch],histtype='step',linewidth=2)

    rms_filt_du = rms_filtered[ mask_dict[du] ]

    for ch in range(rms_filt_du.shape[1]):
        median_filt = np.median(rms_filt_du[:,ch])
        plt.hist(rms_filt_du[:,ch],bins=bins,alpha=.5,label=f'{channels[ch]} filtered: median = {np.round(median_filt,decimals=2)}',color=colors[ch])

    plt.yscale('log')
    plt.xlim([0,100])
    plt.xlabel('RMS [ADC counts]')
    plt.ylabel('Counts')


    # title = r'\texttt{' + os.path.basename(root_file) + r'}' + f': DU {du}'
    # plt.title(title,fontsize=13)

    plt.text(55,2.5e1,'GRAND preliminary', color='crimson')

    title = f'DU {du} RMS on {start_time.date()} between {start_time.time()}--{end_time.time()} UTC+8'
    plt.title(title,fontsize=20)

    plt.legend(fontsize=15)

    if savefig:
        plot_name = f'rms_du_{du}_{start_time.date()}_{start_time.time()}' #+ param_str
        
        plt.savefig( os.path.join(plot_dir,plot_name+'.png') )
        plt.savefig( os.path.join(plot_dir,plot_name+'.pdf') )

    plt.show()

In [ ]:
importlib.reload(tools)

npz_file = '/sps/grand/pcorrea/nutrig/datasets/bkg/gp13_raw/GP13_20240301_085053_RUN125_MD_RAW_10s_ChanXYZ_20dB_11DUs_test_137_dat.npz'

f = np.load(npz_file)

tools.get_masks_du(f['du_ids'])

In [ ]:
x = np.arange(20240116,20240131+1)
x = np.hstack( (x, np.arange(20240201,20240229+1) ) )
x = np.hstack( (x, np.arange(20240301,20240331+1) ) )
x = np.hstack( (x, np.arange(20240401,20240430+1) ) )

np.hstack((x[::5],x[-1]))

In [ ]:
root_files[1000]

In [ ]:
f = np.load('./GP13_20240301_174613_RUN125_MD_RAW_10s_ChanXYZ_20dB_11DUs_test_144_dat.npz')

In [ ]:
x = np.arange(4)
x = x[:-1]
x

In [ ]:
n_max = 509*1.648e-6*1e3
n_max